# Serialize Python Functions as Docker images

Here a python decorator is created that perform the following steps:
- Serialize the function that is decorated
- Create a docker image around the function

In [168]:
import dill
import os
import tempfile
import docker

# *args, **kwargs

"""
docker_image is an annotation that make a python method runnable in a container
@param image: name of the image to create
"""
def docker_image(image_name):
    
    FUNCTION_FILE = "main.txt"
    MAIN_SCRIPT = "my_script.py"
    
    # Serialize function into root_directory
    def serialize_func(func, root_directory):
        ser = dill.dumps(func)
        f = open(os.path.join(root_directory, FUNCTION_FILE), "wb")
        f.write(ser)
        f.close()
        
    def create_main_script(root_directory):
        my_script = f"""
import dill
f = open('{FUNCTION_FILE}', "rb")
main_function = dill.load(f)
f.close()
main_function()
        """
        f = open(os.path.join(root_directory, MAIN_SCRIPT), "w")
        f.write(my_script)
        f.close()
        
    def create_docker_file(root_directory):
        Dockerfile = f"""
FROM python:3.7.6-alpine3.10
RUN pip install dill
ADD {MAIN_SCRIPT} /
ADD {FUNCTION_FILE} /
ENTRYPOINT [ "python3", "./{MAIN_SCRIPT}" ]
        """
        f = open(os.path.join(root_directory, "Dockerfile"), "w")
        f.write(Dockerfile)
        f.close()
        
    def docker_build(client, image_name, root_directory):
        client.images.build(
            path=root_directory, 
            rm=True, 
            tag=image_name)
        
    def docker_push(client, image_name):
        client.images.push(image_name, stream=True, decode=True)
    
    def inner(func): 
        root_directory = tempfile.mkdtemp()
        serialize_func(func, root_directory)
        create_main_script(root_directory)
        create_docker_file(root_directory)
        client = docker.DockerClient(base_url='unix://var/run/docker.sock')
        docker_build(client, image_name, root_directory)
        docker_push(client, image_name)
        
    return inner

In [182]:
@docker_image(image_name = "localhost:32000/library/volume") 
def func():
    import sys

    print("This is the name of the script: ", sys.argv[0])
    print("Number of arguments: ", len(sys.argv))
    print("The arguments are: " , str(sys.argv))  
    print("Inside actual function")
    
    import glob
    print(glob.glob("/mounting/*"))

In [184]:
!docker run -v $(pwd):/mounting localhost:32000/library/volume Hello!

This is the name of the script:  ./my_script.py
Number of arguments:  2
The arguments are:  ['./my_script.py', 'Hello!']
Inside actual function
['/mounting/run-notebook.sh', '/mounting/5 - Workflow - Argo With Parameters.ipynb', '/mounting/README.md', '/mounting/test.yaml', '/mounting/4 - Workflow - DAG.ipynb', '/mounting/3 - Workflow - Single Task.ipynb', '/mounting/Main.ipynb', '/mounting/Serialize Functions.ipynb', '/mounting/rmi-advanced.ipynb', '/mounting/FreeSurfer Executor.ipynb', '/mounting/1 - Minio Connection.ipynb', '/mounting/data', '/mounting/endpoint.yaml', '/mounting/LICENSE', '/mounting/ubuntu.txt', '/mounting/argo-test.yaml', '/mounting/test-pvc.yaml', '/mounting/rmi.ipynb', '/mounting/2 - Create Docker Image.ipynb', '/mounting/service.yaml', '/mounting/license.txt', '/mounting/minio-deployment.yaml']
